In [37]:
import socket
import pickle
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import padding
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [38]:
server_ip = 'localhost'
server_port = 5000
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind((server_ip, server_port))
server_socket.listen(3)  # Listening to 3 clients

print(f"Server listening on {server_ip}:{server_port}")

Server listening on localhost:5000


In [39]:
server_private_key = ec.generate_private_key(ec.SECP256R1())
server_public_key = server_private_key.public_key()

In [40]:
client_connections = []
for i in range(3):
    conn, addr = server_socket.accept()
    print(f"Client {i+1} connected from {addr}")
    client_connections.append(conn)
    conn.send(server_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    ))

Client 1 connected from ('127.0.0.1', 51855)
Client 2 connected from ('127.0.0.1', 51856)
Client 3 connected from ('127.0.0.1', 51858)


In [41]:
client_public_keys = []
for i in range(3):
    client_public_key_pem = client_connections[i].recv(1024)
    client_public_key = serialization.load_pem_public_key(client_public_key_pem)
    client_public_keys.append(client_public_key)

In [42]:

global_weights = None
def build_model():
    model = Sequential([
        Dense(16, activation='relu', input_shape=(13,)),
        Dense(8, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [43]:
model = build_model()


num_rounds = 200
for round_num in range(num_rounds):
    print(f"Starting round {round_num + 1}/{num_rounds}")
    
    client_weights = []
    

    for i in range(3):
        data = pickle.dumps(global_weights)
   
        shared_key = server_private_key.exchange(ec.ECDH(), client_public_keys[i])
        derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data').derive(shared_key)
        aesgcm = AESGCM(derived_key)
        nonce = b'\x00' * 12  
        encrypted_data = aesgcm.encrypt(nonce, data, None)
        client_connections[i].send(encrypted_data)
    

    for i in range(3):
        encrypted_data = client_connections[i].recv(4096)
        shared_key = server_private_key.exchange(ec.ECDH(), client_public_keys[i])
        derived_key = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data').derive(shared_key)
        aesgcm = AESGCM(derived_key)
        data = aesgcm.decrypt(nonce, encrypted_data, None)
        local_weights = pickle.loads(data)
        client_weights.append(local_weights)
    

    new_weights = []
    for weights_list in zip(*client_weights):
        new_weights.append(np.mean(weights_list, axis=0))
    
    global_weights = new_weights
    model.set_weights(global_weights)
    
    print(f"Round {round_num + 1} complete.")



model.save('federated_model.h5')
import pandas as pd
data=pd.read_csv('heart3ex.csv')
X_test=data.drop('target',axis=1)
y_test=data['target']
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Global Model Accuracy: {accuracy * 100:.2f}%")

for conn in client_connections:
    conn.close()

print("Federated learning complete and connections closed.")


Starting round 1/200


c:\Users\krish\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Round 1 complete.
Starting round 2/200
Round 2 complete.
Starting round 3/200
Round 3 complete.
Starting round 4/200
Round 4 complete.
Starting round 5/200
Round 5 complete.
Starting round 6/200
Round 6 complete.
Starting round 7/200
Round 7 complete.
Starting round 8/200
Round 8 complete.
Starting round 9/200
Round 9 complete.
Starting round 10/200
Round 10 complete.
Starting round 11/200
Round 11 complete.
Starting round 12/200
Round 12 complete.
Starting round 13/200
Round 13 complete.
Starting round 14/200
Round 14 complete.
Starting round 15/200
Round 15 complete.
Starting round 16/200
Round 16 complete.
Starting round 17/200
Round 17 complete.
Starting round 18/200
Round 18 complete.
Starting round 19/200
Round 19 complete.
Starting round 20/200
Round 20 complete.
Starting round 21/200
Round 21 complete.
Starting round 22/200
Round 22 complete.
Starting round 23/200
Round 23 complete.
Starting round 24/200
Round 24 complete.
Starting round 25/200
Round 25 complete.
Starting round

Round 200 complete.
Global Model Accuracy: 54.46%
Federated learning complete and connections closed.
